# 한빛출판네트워크 새로나온 책 Crawling

In [1]:
from bs4 import BeautifulSoup 
import urllib.request
import pandas as pd 

In [2]:
base_url = 'https://www.hanbit.co.kr'
sub_url = '/academy/books/new_book_list.html'
url = base_url + sub_url
with urllib.request.urlopen(url) as response:
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')

In [3]:
lis = soup.select('.sub_book_list')
len(lis)

20

In [4]:
lis[6]

<li class="sub_book_list">
<div class="view_box">
<span class="view_box_block">
<span class="img_hidden_a">
<span class="pop_quick_menu">
<a href="/academy/books/book_view.html?p_code=B5747914873"><img alt="상세보기" src="https://www.hanbit.co.kr/images/common/icon_view_detail.png"/></a>
<a href="javascript:;" onclick="addCart('B5747914873','','1','');">
<img alt="장바구니 담기" src="https://www.hanbit.co.kr/images/common/icon_add_cart.png"/>
</a>
</span>
<span class="price">43,000<span>원</span></span>
<span class="pop_quick_bg" onclick="location='/academy/books/book_view.html?p_code=B5747914873'"></span>
</span>
<!-- 예약도서인 경우 -->
<!-- ebook이 도서인 경우 -->
<img alt="" class="thumb" src="/data/books/B5747914873_m.jpg"/>
</span>
<div class="info">
<p class="book_tit"><a href="/academy/books/book_view.html?p_code=B5747914873">STEM CookBook, 대학물리학(5판)</a></p>
<p class="book_writer">Alan Giambattista </p>
</div>
</div>
</li>

In [5]:
book_url = lis[6].find('a').attrs['href']
book_url

'/academy/books/book_view.html?p_code=B5747914873'

In [6]:
with urllib.request.urlopen(base_url+book_url) as response:
    html = response.read()
    book_soup = BeautifulSoup(html, 'html.parser')

In [7]:
info = book_soup.select_one('.store_product_info_box')
title = info.find('h3').string
title

'STEM CookBook, 대학물리학(5판)'

In [8]:
lis = info.find_all('li')
book_dict = {}
for li in lis:
    idx = li.find('strong').get_text().find(':')
    key = li.find('strong').get_text()[:idx-1]
    value = li.find('span').get_text()
    book_dict.update([[key, value]])
book_dict

{'저자': 'Alan Giambattista ',
 '번역': '김용은 외 ',
 '출간': '2021-01-20',
 '페이지': '1464 쪽',
 'ISBN': '9791156645207',
 '물류코드': '4520',
 '원제': 'College Physics, 5th Edition(McGraw-Hill Education)'}

In [9]:
al = 'Behrouz A. Forouzan , Richard F. Gilberg '.split(',')
al = list(map(lambda s: s.rstrip(), al))
','.join(al)

'Behrouz A. Forouzan, Richard F. Gilberg'

In [10]:
info = book_soup.select_one('.store_product_info_box')
book_lis = info.find_all('li')
al = book_lis[0].find('span').get_text().split(',')
authors = ','.join(list(map(lambda s: s.rstrip(), al)))

for bli in book_lis[1:]:
    key = bli.find('strong').get_text()
    if key.find('번역') >= 0:
        tl = bli.find('span').get_text().split(',')
        translators = ','.join(list(map(lambda s: s.rstrip(), tl)))
    elif key.find('출간') >= 0:
        date = bli.find('span').get_text()
    elif key.find('페이지') >= 0:
        page = int(bli.find('span').get_text()[:-2])
    elif key.find('ISBN') >= 0:
        isbn = bli.find('span').get_text()

print(authors, translators, date, page, isbn)

Alan Giambattista 김용은 외 2021-01-20 1464 9791156645207


In [11]:
pb_ps = book_soup.select_one('.payment_box').find_all('p')
list_prices = []; sales_prices = []; mileages = []

idx = pb_ps[0].select_one('.pbr').get_text().find('원')
lp = pb_ps[0].select_one('.pbr').get_text()[:idx]
lp = int(lp.replace(',',''))

idx = pb_ps[1].select_one('.pbr').get_text().find('원')
sp = pb_ps[1].select_one('.pbr').get_text()[:idx]
sp = int(sp.replace(',',''))

idx = pb_ps[2].select_one('.pbr').get_text().find('점')
mile = pb_ps[2].select_one('.pbr').get_text()[:idx]
mile = int(mile.replace(',',''))

print(lp, sp, mile)

43000 43000 1290


#### 첫번째 페이지

In [12]:
title_list = []; info_list = []; pb_list = []
lis = soup.select('.sub_book_list')
for li in lis:
    book_url = li.find('a').attrs['href']
    with urllib.request.urlopen(base_url+book_url) as response:
        html = response.read()
        book_soup = BeautifulSoup(html, 'html.parser')
    info = book_soup.select_one('.store_product_info_box')
    title = info.find('h3').string
    title_list.append(title)

    book_lis = info.find_all('li')
    book_dict = {}
    for book_li in book_lis:
        idx = book_li.find('strong').get_text().find(':')
        key = book_li.find('strong').get_text()[:idx-1]
        value = book_li.find('span').get_text()
        book_dict.update([[key, value]])
    info_list.append(book_dict)

    pb_ps = book_soup.select_one('.payment_box').find_all('p')
    pb_dict = {}
    for pb in pb_ps:
        idx = pb.select_one('.pbl').get_text().find(':')
        key = pb.select_one('.pbl').get_text()[:idx-1]
        value = pb.select_one('.pbr').get_text()
        pb_dict.update([[key, value]])
    pb_list.append(pb_dict)

In [13]:
print(title_list)

['4차 산업혁명 시대를 위한 e-비즈니스 개론(4판)', 'IT CookBook, C 언어 일취월장(2판)', 'Numerical Methods for Engineers, 8th Edition', 'IT CookBook, SQL Server로 배우는 데이터베이스 개론과 실습(2판)', 'IT CookBook, 손에 잡히는 데이터 통신', 'IT CookBook, 파이썬 자료구조와 알고리즘 for Beginner', 'STEM CookBook, 대학물리학(5판)', 'STEM CookBook, 재료역학(2판)', 'IT CookBook, 디지털 회로 실험 with PSpice', '한번에 통과하는 논문 세트', '데이터 과학을 위한 기초수학 with 파이썬', '미분적분학 : 대학수학의 첫걸음', '제대로 시작하는 기초 통계학 : Excel 활용 (2판)', '친절한 미분적분학', 'IT CookBook, 4차 산업혁명과 정보통신의 이해', 'IT CookBook, Android Studio를 활용한 안드로이드 프로그래밍(6판)', 'IT CookBook, C# 프로그래밍 2판', 'IT CookBook, 게임 제작으로 배우는 유니티', 'IT CookBook, 난생처음 R 코딩 & 데이터 분석', 'IT CookBook, 난생처음 인공지능 with 스크래치']


In [14]:
info_list

[{'저자': '이동훈 ',
  '출간': '2021-03-07',
  '페이지': '512 쪽',
  'ISBN': '9791156645320',
  '물류코드': '4532'},
 {'저자': '신윤환 ',
  '출간': '2021-02-24',
  '페이지': '800 쪽',
  'ISBN': '9791156645313',
  '물류코드': '4531',
  '구판정보': '이 도서는 <IT CookBook, C 언어 일취월장>의 개정판입니다. 구판 정보 보기'},
 {'저자': 'Steven Chapra , Raymond Canale ',
  '출간': '2021-02-18',
  '페이지': '1008 쪽',
  'ISBN': '9781260571387',
  '물류코드': '30003'},
 {'저자': '박우창 , 남송휘 , 이현룡 ',
  '출간': '2021-02-10',
  '페이지': '604 쪽',
  'ISBN': '9791156645290',
  '물류코드': '4529',
  '구판정보': '이 도서는 <IT CookBook, SQL Server로 배우는 데이터베이스 개론과 실습>의 개정판입니다. 구판 정보 보기'},
 {'저자': '임석구 ',
  '출간': '2021-01-23',
  '페이지': '572 쪽',
  'ISBN': '9791156645283',
  '물류코드': '4528'},
 {'저자': '우재남 ',
  '출간': '2021-01-23',
  '페이지': '520 쪽',
  'ISBN': '9791156645252',
  '물류코드': '4525'},
 {'저자': 'Alan Giambattista ',
  '번역': '김용은 외 ',
  '출간': '2021-01-20',
  '페이지': '1464 쪽',
  'ISBN': '9791156645207',
  '물류코드': '4520',
  '원제': 'College Physics, 5th Edition(McGraw-Hill Education)'},
 {'저자

In [15]:
pb_list

[{'정가': '27,000원', '판매가': '27,000원(0% off)', '마일리지': '810점 (3%)'},
 {'정가': '28,000원', '판매가': '28,000원(0% off)', '마일리지': '840점 (3%)'},
 {'정가': '45,000원', '판매가': '45,000원(0% off)', '마일리지': '1,350점 (3%)'},
 {'정가': '29,000원', '판매가': '29,000원(0% off)', '마일리지': '870점 (3%)'},
 {'정가': '29,000원', '판매가': '29,000원(0% off)', '마일리지': '870점 (3%)'},
 {'정가': '27,000원', '판매가': '27,000원(0% off)', '마일리지': '810점 (3%)'},
 {'정가': '43,000원', '판매가': '43,000원(0% off)', '마일리지': '1,290점 (3%)'},
 {'정가': '38,000원', '판매가': '38,000원(0% off)', '마일리지': '1,140점 (3%)'},
 {'정가': '22,000원', '판매가': '22,000원(0% off)', '마일리지': '660점 (3%)'},
 {'정가': '85,000원', '판매가': '85,000원(0% off)', '마일리지': '2,550점 (3%)'},
 {'정가': '26,000원', '판매가': '26,000원(0% off)', '마일리지': '780점 (3%)'},
 {'정가': '28,000원', '판매가': '28,000원(0% off)', '마일리지': '840점 (3%)'},
 {'정가': '25,000원', '판매가': '25,000원(0% off)', '마일리지': '750점 (3%)'},
 {'정가': '32,000원', '판매가': '32,000원(0% off)', '마일리지': '960점 (3%)'},
 {'정가': '25,000원', '판매가': '25,000원(0% off)', '마일리지': '

In [16]:
title_list = []; author_list = []; translator_list = []
date_list = []; page_list = []; isbn_list = []
list_price_list = []; sales_price_list = []; mileage_list = []

for li in lis:
    book_url = li.find('a').attrs['href']
    with urllib.request.urlopen(base_url+book_url) as response:
        html = response.read()
        book_soup = BeautifulSoup(html, 'html.parser')
    info = book_soup.select_one('.store_product_info_box')
    title = info.find('h3').string
    title_list.append(title)

    book_lis = info.find_all('li')
    al = book_lis[0].find('span').get_text().split(',')
    authors = ','.join(list(map(lambda s: s.rstrip(), al)))
    author_list.append(authors)

    tr_flag = False
    for bli in book_lis[1:]:
        key = bli.find('strong').get_text()
        if key.find('출간') >= 0:
            date = bli.find('span').get_text()
            date_list.append(date)
            continue
        elif key.find('페이지') >= 0:
            page = int(bli.find('span').get_text()[:-2])
            page_list.append(page)
            continue
        elif key.find('ISBN') >= 0:
            isbn = bli.find('span').get_text()
            isbn_list.append(isbn)
            continue
        elif key.find('번역') >= 0:
            tl = bli.find('span').get_text().split(',')
            translators = ','.join(list(map(lambda s: s.rstrip(), tl)))
            translator_list.append(translators)
            tr_flag = True
    
    if not tr_flag:
        translator_list.append(' ')

    pb_ps = book_soup.select_one('.payment_box').find_all('p')
    idx = pb_ps[0].select_one('.pbr').get_text().find('원')
    lp = pb_ps[0].select_one('.pbr').get_text()[:idx]
    lp = int(lp.replace(',',''))
    list_price_list.append(lp)

    idx = pb_ps[1].select_one('.pbr').get_text().find('원')
    sp = pb_ps[1].select_one('.pbr').get_text()[:idx]
    sp = int(sp.replace(',',''))
    sales_price_list.append(sp)

    idx = pb_ps[2].select_one('.pbr').get_text().find('점')
    mile = pb_ps[2].select_one('.pbr').get_text()[:idx]
    mile = int(mile.replace(',',''))
    mileage_list.append(mile)

In [17]:
len(title_list), len(author_list), len(translator_list), len(date_list), len(page_list), len(isbn_list), len(list_price_list), len(sales_price_list), len(mileage_list)

(20, 20, 20, 20, 20, 20, 20, 20, 20)

In [18]:
new_book = pd.DataFrame({
            '제목': title_list,
            '저자': author_list,
            '번역': translator_list,
            '출간일': date_list,
            '페이지': page_list,
            'ISBN': isbn_list,
            '정가': list_price_list,
            '판가': sales_price_list,
            '마일리지': mileage_list
})
new_book

,제목,저자,번역,출간일,페이지,ISBN,정가,판가,마일리지
0,4차 산업혁명 시대를 위한 e-비즈니스 개론(4판),이동훈,,2021-03-07,512,9791156645320,27000,27000,810
1,"IT CookBook, C 언어 일취월장(2판)",신윤환,,2021-02-24,800,9791156645313,28000,28000,840
2,"Numerical Methods for Engineers, 8th Edition","Steven Chapra, Raymond Canale",,2021-02-18,1008,9781260571387,45000,45000,1350
3,"IT CookBook, SQL Server로 배우는 데이터베이스 개론과 실습(2판)","박우창, 남송휘, 이현룡",,2021-02-10,604,9791156645290,29000,29000,870
4,"IT CookBook, 손에 잡히는 데이터 통신",임석구,,2021-01-23,572,9791156645283,29000,29000,870
5,"IT CookBook, 파이썬 자료구조와 알고리즘 for Beginner",우재남,,2021-01-23,520,9791156645252,27000,27000,810
6,"STEM CookBook, 대학물리학(5판)",Alan Giambattista,김용은 외,2021-01-20,1464,9791156645207,43000,43000,1290
7,"STEM CookBook, 재료역학(2판)","Anthony Bedford, Kenneth M. Liechti","정광영, 김형종, 마정범, 최정호, 홍석무",2021-01-20,840,9791156645191,38000,38000,1140
8,"IT CookBook, 디지털 회로 실험 with PSpice",홍순관,,2021-01-15,396,9791156645122,22000,22000,660
9,한번에 통과하는 논문 세트,히든그레이스 논문통계팀,,2021-01-08,1448,9791156645177,85000,85000,2550


### 모든 페이지에 대해서 처리

In [20]:
import tqdm

In [21]:
base_url = 'https://www.hanbit.co.kr'
sub_url = '/academy/books/new_book_list.html'
title_list = []; author_list = []; translator_list = []
date_list = []; page_list = []; isbn_list = []
list_price_list = []; sales_price_list = []; mileage_list = []
total_page = 17

In [22]:
for page in tqdm.notebook.tqdm(range(total_page)):
    page_url = f'?page={page+1}&cate_cd=&srt=&searchKey=&keyWord='
    url = base_url + sub_url + page_url
    with urllib.request.urlopen(url) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')

    lis = soup.select('.sub_book_list')
    for li in lis:
        book_url = li.find('a').attrs['href']
        with urllib.request.urlopen(base_url+book_url) as response:
            html = response.read()
            book_soup = BeautifulSoup(html, 'html.parser')
        info = book_soup.select_one('.store_product_info_box')
        title = info.find('h3').string
        title_list.append(title)

        book_lis = info.find_all('li')
        al = book_lis[0].find('span').get_text().split(',')
        authors = ','.join(list(map(lambda s: s.rstrip(), al)))
        author_list.append(authors)

        tr_flag = False
        for bli in book_lis[1:]:
            key = bli.find('strong').get_text()
            if key.find('출간') >= 0:
                date = bli.find('span').get_text()
                date_list.append(date)
                continue
            elif key.find('페이지') >= 0:
                page = int(bli.find('span').get_text()[:-2])
                page_list.append(page)
                continue
            elif key.find('ISBN') >= 0:
                isbn = bli.find('span').get_text()
                isbn_list.append(isbn)
                continue
            elif key.find('번역') >= 0:
                tl = bli.find('span').get_text().split(',')
                translators = ','.join(list(map(lambda s: s.rstrip(), tl)))
                translator_list.append(translators)
                tr_flag = True
        
        if not tr_flag:
            translator_list.append(' ')

        pb_ps = book_soup.select_one('.payment_box').find_all('p')
        idx = pb_ps[0].select_one('.pbr').get_text().find('원')
        lp = pb_ps[0].select_one('.pbr').get_text()[:idx]
        lp = int(lp.replace(',',''))
        list_price_list.append(lp)

        idx = pb_ps[1].select_one('.pbr').get_text().find('원')
        sp = pb_ps[1].select_one('.pbr').get_text()[:idx]
        sp = int(sp.replace(',',''))
        sales_price_list.append(sp)

        idx = pb_ps[2].select_one('.pbr').get_text().find('점')
        mile = pb_ps[2].select_one('.pbr').get_text()[:idx]
        mile = int(mile.replace(',',''))
        mileage_list.append(mile)

<ipython-input-22-795402cb808f>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for page in tqdm_notebook(range(total_page)):


  0%|          | 0/17 [00:00<?, ?it/s]

In [23]:
new_book = pd.DataFrame({
            '제목': title_list,
            '저자': author_list,
            '번역': translator_list,
            '출간일': date_list,
            '페이지': page_list,
            'ISBN': isbn_list,
            '정가': list_price_list,
            '판가': sales_price_list,
            '마일리지': mileage_list
})
new_book.tail()

,제목,저자,번역,출간일,페이지,ISBN,정가,판가,마일리지
317,문제해결력을 키우는 이공계 글쓰기 : 발표와 토론까지,한성우,,2013-02-28,396,9788998756024,18000,18000,540
318,"진로개발과 직업세계 : 대학생과 청년층을 위한 진로, 취업지침서","서일수, 김도균, 김지혜, 이정배",,2013-02-28,392,9788998756048,20000,20000,600
319,Essential Calculus,James Stewart,,2012-07-08,960,9781133492566,36000,36000,1080
320,"IT CookBook, 디지털 통신 시스템 : Digital Communicatio...","정호영, 임승각",,2011-01-31,576,9788979147988,25000,25000,750
321,"IT CookBook, 운영체제론 : Operating Systems, 3rd Ed","하비 디텔, 폴 디텔, 데이빗 쇼픈스",송경희,2009-09-04,860,9788979146844,32000,32000,960


In [24]:
new_book.to_csv('New_Book_List.tsv', sep='\t', encoding='utf8', index=False)